# Create OGSL table
Create a DataFrame that associates every sign value that is recognized in [ORACC](http://oracc.org) with a unicode code point (or a sequence of unicode code points). The table may be used to translate a text in transliteration into a sequence of unicode code points for use in `fasttext`. The data are derived from the ORACC Global Sign List ([OGSL](http://build-oracc.museum.upenn.edu)).


In [ ]:
import pandas as pd
import zipfile
import json
import tqdm
import requests
import errno
import os
import pickle

## 0 Create Directories, if Necessary
The two directories needed for this script are `jsonzip` and `output`. If they do not exist they are created, else: do nothing.

For the code, see [Stack Overflow](http://stackoverflow.com/questions/18973418/os-mkdirpath-returns-oserror-when-directory-does-not-exist).

In [ ]:
directories = ['jsonzip', 'output']
for d in directories:
    try:
        os.mkdir(d)
    except OSError as exc:
        if exc.errno !=errno.EEXIST:
            raise
        pass

## 1 Download the ZIP file

In [ ]:
project = "ogsl"
CHUNK = 16 * 1024
url = "http://build-oracc.museum.upenn.edu/json/" + project + ".zip"
file = 'jsonzip/' + project + '.zip'
r = requests.get(url)
if r.status_code == 200:
    print("Downloading " + url + " saving as " + file)
    with open(file, 'wb') as f:
        for c in r.iter_content(chunk_size=CHUNK):
            f.write(c)
else:
    print(url + " does not exist.")

# <a name="head21"></a>2 The `parsejson()` function

In [ ]:
def parsejson(data_json):
    for key, value in data_json["signs"].items():
        if "values" in value:
            for n in value["values"]:
                s["value"] = n
                s["name"] = key
                if "utf8" in value:
                    s["utf8"] = value["utf8"]
                    s["hex"] = value["hex"] 
                else:
                    s["utf8"] = ""
                    s["hex"] = ""
                sign = {key : value for key, value in s.items()}
                s_l.append(sign)
    return

# 3 Main Process

In [ ]:
s = {}
s_l = []
file = "jsonzip/ogsl.zip"
z = zipfile.ZipFile(file) 
filename = "ogsl/ogsl-sl.json"
signlist = z.read(filename).decode('utf-8')
data_json = json.loads(signlist)                # make it into a json object (essentially a dictionary)
parsejson(data_json)  

# 4 Make Dataframe

In [ ]:
df = pd.DataFrame(s_l)
df

In [ ]:
with open("output/ogsl.p", "wb") as p:
    pickle.dump(df, p)